# OpenAI Academy

[![Static Badge](https://img.shields.io/badge/Open%20in%20Modal%20-%20Agents?style=for-the-badge&color=green)](https://modal.com/notebooks/new/https://github.com/CelestoAI/agentor/blob/main/examples/openai-academy.ipynb)


## Agents with examples

In [1]:
# %uv pip install openai-agents==0.3.3 agentor

## What is an Agent?

<img src="https://i.imgflip.com/a7rr5a.jpg" width="250px">

<br>

## Agents are LLMs with higher autonomy

- Tool access
- Memory

In [2]:
from dotenv import load_dotenv
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="Write a haiku about recursion in programming."
)

print(response.output[-1].content[0].text) 

Calls itself again
until the base case ends, then
returns to the call


In [3]:
import datetime as dt

from agents import Agent, Runner

load_dotenv()

agent = Agent(name="Assistant", instructions="You are a helpful assistant", model="gpt-5-nano")

result = await Runner.run(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

Recursive whispers
Until a base case is found
Or it ends again


In [4]:
agent = Agent(name="Assistant", instructions="Reply very concisely.")

result = await Runner.run(agent, "What city is the Golden Gate Bridge in?")
print(result.final_output)

San Francisco.


# Tool use

In [5]:
result = await Runner.run(agent, "What is the weather in London?")
print(result.final_output)

I can’t provide real-time weather updates. Please check a weather website or app for current London weather.


In [6]:
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

In [7]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-5-nano",
    tools=[get_weather],
)

In [8]:
result = await Runner.run(agent, "What is the weather in London?")
print(result.final_output)

London wears bright sun
cheerful streets glow in sunlight
A warm hopeful day


## LLM with tool use

In [9]:
weather_tool_def = {
    "type": "function",
    "name": "get_weather",
    "description": "Retrieves current weather for the given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. London, United Kingdom"
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Units the temperature will be returned in."
            }
        },
        "required": ["location", "units"],
        "additionalProperties": False
    },
    "strict": True
}

def get_weather(location: str, units: str) -> str:
    """returns weather info for the specified location and units."""
    return f"The weather in {location} is sunny with a temperature of 20 degrees {units}."

tool_registry = {
    "get_weather": get_weather
}


In [10]:
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="What is the weather in London?",
    tools=[weather_tool_def]
)


In [11]:
print(response.output[-1])

ResponseFunctionToolCall(arguments='{"location":"London, United Kingdom","units":"celsius"}', call_id='call_YXUqiJlBOMt4KePCZUf4Tswg', name='get_weather', type='function_call', id='fc_0cba6b48de09cff20068e3a7f5c69c8190a00722ef52dfb7d6', status='completed')


In [12]:
import json

if response.output[-1].type == "function_call":
    fn_name = response.output[-1].name
    arguments = json.loads(response.output[-1].arguments)
    result = tool_registry[fn_name](**arguments)
    print(result)
else:
    print(response.output[-1].content[0].text)

The weather in London, United Kingdom is sunny with a temperature of 20 degrees celsius.


# Scheduling Agent

We will build an Agent with access to email and calendar to help schedule meetings on behalf of the user.

**Example conversation:**
- Lock wants a meeting
- Smith proposes a call
- AI assistant handles scheduling automatically

## Gmail and Calendar API

In [15]:
from agentor.integrations.google import GmailService, CalendarService, load_user_credentials
from rich import print

# Load your saved credentials
creds = load_user_credentials("credentials.my_google_account.json")

gmail = GmailService(creds)
calendar = CalendarService(creds)

In [ ]:
print("List emails", gmail.list_messages(limit=2))
print("Get an email", gmail.get_message(message_id="199a56838f299758"))
print("Get full email body", gmail.get_message_body(message_id="199a56838f299758"))

In [ ]:
print("Search emails", gmail.search_messages(query="from:aniket"))

In [ ]:
print("List calendar events", calendar.list_events())

In [ ]:
event = {
    "summary": "OpenAI Academy - Agents SDK Session with Aniket",
    "description": "I attended the OpenAI Academy live session on Agents SDK with Aniket",
    "start": dt.datetime(2025, 10, 6, 15, 0, tzinfo=dt.timezone.utc),
    "end": dt.datetime(2025, 10, 6, 16, 0, tzinfo=dt.timezone.utc),
    "attendees": [{"email": "aniket@example.com"}],
}
calendar.create_event(event=event)

# Multi-Agent System

Agent which interacts with user to schedule a meeting.

In [20]:
from openai.types.responses import ResponseTextDeltaEvent
from agents import ItemHelpers

In [21]:
def view_availability(start: dt.datetime, end: dt.datetime) -> str:
    return calendar.list_events(time_min=start, time_max=end)["events"]

availability = view_availability(dt.datetime(2025, 10, 6, 15, 0, tzinfo=dt.timezone.utc), dt.datetime(2025, 10, 6, 16, 0, tzinfo=dt.timezone.utc))[0]
print(f"{availability['start']['dateTime']} is booked for '{availability['summary']}'")

2025-10-06T16:00:00+01:00 is booked for 'OpenAI Academy - Agents SDK Session with Aniket'

In [ ]:
def book_meeting(summary:str, description:str, start: dt.datetime, end: dt.datetime, attendees: list[str]) -> str:
    event = {
        "summary": summary,
        "description": description,
        "start": start,
        "end": end,
        "attendees": attendees,
        }
    output = calendar.create_event(event=event)
    return f"Meeting booked successfully. Details: {output['event']}"

book_meeting(summary="OpenAI Academy - Agents SDK Session with Aniket", description="I attended the OpenAI Academy live session on Agents SDK with Aniket", start=dt.datetime(2025, 10, 6, 15, 0, tzinfo=dt.timezone.utc), end=dt.datetime(2025, 10, 6, 16, 0, tzinfo=dt.timezone.utc), attendees=["aniket@example.com"])

In [23]:
@function_tool
def view_availability(start: dt.datetime, end: dt.datetime) -> str:
    """Returns a list of events that overlap with the given time range. The output contains events booked on the calendar."""
    return calendar.list_events(time_min=start, time_max=end)["events"]


@function_tool
def book_meeting(summary:str, description:str, start: dt.datetime, end: dt.datetime, attendees: list[str]) -> str:
    """Books a meeting on the calendar. The output contains the event details.
    
    Args:
        summary: Summary of the meeting
        description: Description of the meeting
        start: Start time of the meeting
        end: End time of the meeting
        attendees: List of attendees emails
    """
    event = {
        "summary": summary,
        "description": description,
        "start": start,
        "end": end,
        "attendees": attendees,
        }
    output = calendar.create_event(event=event)
    return f"Meeting booked successfully. Details: {output['event']}"

In [24]:
calendar_agent = Agent(
    name="Calendar Agent",
    instructions="You are a helpful assistant that can help schedule meetings on the calendar. You use sensible defaults for the meeting details.",
    model="gpt-5-mini",
    tools=[view_availability, book_meeting],
)


In [ ]:
result = Runner.run_streamed(calendar_agent, "I want to schedule a meeting with Aniket (aniket@example.com) on 6th October 2025 from 15:00 to 16:00 BST")

async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

## Orchestrating multiple agents


**Example:**
- Schedule meeting
- Send email reminders

### Handoff Agents

In [26]:
calendar_agent = Agent(
    name="Calendar Agent",
    instructions="You are a helpful assistant that can help schedule meetings on the calendar. You use sensible defaults for the meeting details.",
    model="gpt-5-mini",
    tools=[view_availability, book_meeting],
)

@function_tool
def send_email(to: str, subject: str, body: str) -> str:
    """Sends an email to the specified recipient. The output contains the email details."""
    return f"Email sent successfully. Details: {to}, {subject}, {body}"

email_agent = Agent(
    name="Email Agent",
    instructions="You are a helpful assistant that can send emails. You use sensible defaults for the email content.",
    model="gpt-5-mini",
    tools=[send_email],
)


triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the user's request.",
    handoffs=[email_agent, calendar_agent],
)


In [27]:
from agents import run_demo_loop
await run_demo_loop(triage_agent)

## Agents as tools


In [28]:
calendar_agent = Agent(
    name="Calendar Agent",
    instructions="You are a helpful assistant that can help schedule meetings on the calendar. You use sensible defaults for the meeting details.",
    model="gpt-5-mini",
    tools=[view_availability, book_meeting],
)

@function_tool
def send_email(to: str, subject: str, body: str) -> str:
    """Sends an email to the specified recipient. The output contains the email details."""
    return f"Email sent successfully. Details: {to}, {subject}, {body}"

email_agent = Agent(
    name="Email Agent",
    instructions="You are a helpful assistant that can send emails. You use sensible defaults for the email content.",
    model="gpt-5-mini",
    tools=[send_email],
)


triage_agent = Agent(
    name="triage_agent",
    instructions="Use the tools provided to help the user with their request.",
    tools = [
        email_agent.as_tool(tool_name="send_email", tool_description="Send an email to the specified recipient. The output contains the email details."), 
        calendar_agent.as_tool(tool_name="calendar_manager", tool_description="Schedule a meeting on the calendar. The output contains the event details.")
        ]
)

from agents import run_demo_loop
await run_demo_loop(triage_agent)